# Proeycto de IA - ChatBox

Departamento de Ciencias de la Computación, Universidad de Chile.

CC6409: Taller de Desarrollo de Poryectod e IA - Otoño 2023

**Integrantes:**
- Garrido Martín
- Gómez Nahuel
- Santelices Gustavo


## Import

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report

from sklearn.base import BaseEstimator, TransformerMixin
import re
import numpy as np

## data

In [3]:
# handmade
hand_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_nawel.csv", sep = ';')
hand_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_nawel.csv", sep = ';')
hand_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_nawel.csv", sep = ';')
hand_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_nawel.csv", sep = ';')

hand_tavo_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_tavo.csv", sep = ';')
hand_tavo_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_tavo.csv", sep = ';')
#hand_tavo_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_tavo.csv", sep = ',')
hand_tavo_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_tavo.csv", sep = ';')

hand_martin_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/asistire_martin.csv", sep = ';', encoding = 'latin-1')
hand_martin_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/no_asistire_martin.csv", sep = ';')
#hand_martin_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/reagendar_martin.csv", sep = ',')
hand_martin_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/Handmade/pedir_martin.csv", sep = ';')

In [4]:
# GPT
gpt_nawel_0 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/asistire_nawel.csv", sep = ';')
gpt_nawel_1 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/no_asistire_nawel.csv", sep = ';')
gpt_nawel_2 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/reagendar_nawel.csv", sep = ';')
gpt_nawel_3 = pd.read_csv("https://raw.githubusercontent.com/QwagPerson/ProyectoDeIA/main/Data/GPT_generated/pedir_nawel.csv", sep = ';')


In [6]:
df_ = pd.concat([hand_nawel_0, hand_nawel_1, hand_nawel_2, hand_nawel_3,
                 gpt_nawel_0, gpt_nawel_1, gpt_nawel_2, gpt_nawel_3,
                 hand_tavo_0, hand_tavo_1, hand_tavo_3,
                 hand_martin_0, hand_martin_1, hand_martin_3])

df_.count()

class    358
text     358
dtype: int64

In [7]:
df_['class'].value_counts()

0    117
1    113
3     91
2     37
Name: class, dtype: int64

## Custom Features

In [8]:
class PreProccesingTransformer(BaseEstimator, TransformerMixin):
    def preprocess(self,sentence):
      # Deleting all except: exclamation/question signs and accents
      new_word = re.sub(r"[^a-zA-ZáéíóúÁÉÍÓÚñÑ¡!¿?\s]", '', sentence)
      # Deleting double blank spaces
      new_sentence = new_word.replace('  ',' ').replace('\n','').strip()
      return new_sentence

    def transform(self, X, y=None):
        values = []
        for tweet in X:
            values.append(self.preprocess(tweet))

        return(np.array(values))

    def fit(self, X, y=None):
        return self

## Pipeline

### Run Pipeline

In [45]:
# Defining threshold
thresholds = {
    0: 0.85,  # class 0
    1: 0.85,  # class 1
    2: 0.85,   # class 2
    3: 0.85   #class 3
}


def custom_predict(model, X, class_thresholds):
    # Get probabilities
    probabilities = model.predict_proba(X)

    # Aplied threshold
    modified_predictions = np.argmax(probabilities, axis=1)
    for class_label, threshold in class_thresholds.items():
        modified_predictions[probabilities[:, class_label] > threshold] = class_label

    return modified_predictions

In [48]:
def run(dataset, pipeline):

    X_train, X_test, y_train, y_test = train_test_split(
        dataset['text'],
        dataset['class'],
        shuffle=True,
        test_size=0.2,
        random_state=42,
        stratify=dataset['class']
    )

    print(f"# Len Training Data: {len(X_train)}")
    print(f"# Len Testing Data: {len(X_test)}")

    pipeline.fit(X_train, y_train)


    # Classes propbailities
    probabilities = pipeline.predict_proba(X_test)


    # Predicted labels
    #predicted_labels = pipeline.predict(X_test)
    predicted_labels = custom_predict(pipeline, X_test, thresholds)


    print(classification_report(y_test, predicted_labels))


    return pipeline, X_train, X_test, y_train, y_test, predicted_labels, probabilities


## Defining Pipelines

In [26]:
# simple Pipeline only BOW

def get_experiment_0_pipeline():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer()),
                    ]
                ),
            ),
            ("clf", MultinomialNB()),
        ]
    )

In [16]:
# simple Pipeline only Preprocessing + BOW

def get_experiment_1_pipeline():

    return Pipeline(
        [
            ("preprocessing", PreProccesingTransformer()),
            (
                "features",
                FeatureUnion(
                    [
                        ("bow", CountVectorizer())
                    ]
                ),
            ),
            ("clf", MultinomialNB()),
        ]
    )

## Implementación del modelo

In [49]:
# Simple Pipeline Test
pipeline0 = get_experiment_0_pipeline()

prob = run(df_, pipeline0)

# Len Training Data: 286
# Len Testing Data: 72
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        24
           1       0.76      0.70      0.73        23
           2       0.75      0.43      0.55         7
           3       0.78      1.00      0.88        18

    accuracy                           0.78        72
   macro avg       0.77      0.73      0.74        72
weighted avg       0.78      0.78      0.77        72



In [50]:
# Simple Pipeline Test (+ preprocessing)
pipeline1 = get_experiment_1_pipeline()

_ = run(df_, pipeline1)

# Len Training Data: 286
# Len Testing Data: 72
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        24
           1       0.76      0.70      0.73        23
           2       0.75      0.43      0.55         7
           3       0.78      1.00      0.88        18

    accuracy                           0.78        72
   macro avg       0.77      0.73      0.74        72
weighted avg       0.78      0.78      0.77        72

